<a href="https://colab.research.google.com/github/abhisheksingh-7/11-785-IDL/blob/main/HW/HW1/P2/HW1P2_S23_Starter_Notebook_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW1: Frame-Level Speech Recognition

In this homework, you will be working with MFCC data consisting of 27 features at each time step/frame. Your model should be able to recognize the phoneme occured in that frame.

# Libraries

In [89]:
!pip install torchsummaryX wandb --quiet

In [90]:
import torch
import numpy as np
from torchsummaryX import summary
import sklearn
import gc
import zipfile
import pandas as pd
from tqdm.auto import tqdm
import os
import datetime
import wandb
import math
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [91]:
### If you are using colab, you can import google drive to save model checkpoints in a folder
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# make directory for homework
import os
import errno
directory_name = '/content/drive/MyDrive/11-785-IDL/HW/HW1/P2'
try:
    os.makedirs(directory_name)
except OSError as exc: 
    if exc.errno == errno.EEXIST and os.path.isdir(directory_name):
        pass

In [ ]:
! ls /content/drive/MyDrive/11-785-IDL/HW/HW1/P2

data


In [92]:
### PHONEME LIST
PHONEMES = [
            '[SIL]',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '[SOS]', '[EOS]']

# Kaggle

This section contains code that helps you install kaggle's API, creating kaggle.json with you username and API key details. Make sure to input those in the given code to ensure you can download data from the competition successfully.

In [93]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!pip install --upgrade --force-reinstall --no-deps kaggle 
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"shekennotstirred","key":"c0878af19b47a450335ad68980815833"}') 
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached kaggle-1.5.8-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached kaggle-1.5.12-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.8
    Uninstalling kaggle-1.5.8:
      Successfully uninstalled kaggle-1.5.8
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [94]:
# commands to download data from kaggle

import os
import errno
directory_name = '/content/drive/MyDrive/11-785-IDL/HW/HW1/P2/data'
try:
    os.makedirs(directory_name)
except OSError as exc: 
    if exc.errno == errno.EEXIST and os.path.isdir(directory_name):
        pass

! cd /content/drive/MyDrive/11-785-IDL/HW/HW1/P2/data
! ls -a 
!kaggle competitions download -c 11-785-s23-hw1p2
!mkdir '/content/data/'
!unzip -qo '11-785-s23-hw1p2.zip' -d '/content/data/'

# !mkdir '/content/drive/MyDrive/11-785-IDL/HW/HW1/P2/data'

# !unzip -qo '11-785-s23-hw1p2.zip' -d '/content/drive/MyDrive/11-785-IDL/HW/HW1/P2/data'

.  ..  .config	data  drive  model_arch.txt  sample_data  submission.csv  wandb
100% 16.0G/16.0G [06:22<00:00, 43.0MB/s]
100% 16.0G/16.0G [06:23<00:00, 44.8MB/s]
mkdir: cannot create directory ‘/content/data/’: File exists


In [ ]:
arr = np.array([[1, 2], [3, 4]])
print(arr)
print(np.mean(arr, axis=0))
arr = arr - np.mean(arr, axis=0)
print(arr)

[[1 2]
 [3 4]]
[2. 3.]
[[-1. -1.]
 [ 1.  1.]]


# Dataset

This section covers the dataset/dataloader class for speech data. You will have to spend time writing code to create this class successfully. We have given you a lot of comments guiding you on what code to write at each stage, from top to bottom of the class. Please try and take your time figuring this out, as it will immensely help in creating dataset/dataloader classes for future homeworks.

Before running the following cells, please take some time to analyse the structure of data. Try loading a single MFCC and its transcipt, print out the shapes and print out the values. Do the transcripts look like phonemes?

In [139]:
# Dataset class to load train and validation data

class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, root, partition, context, phonemes, portion=1): # Feel free to add more arguments

        self.context = context
        self.phonemes = phonemes
        self.root = root
        self.partition = partition
        
        # MFCC directory - use partition to acces train/dev directories from kaggle data using root
        self.mfcc_dir       = self.root + self.partition + '/' + 'mfcc/'
        # Transcripts directory - use partition to acces train/dev directories from kaggle data using root
        self.transcript_dir = self.root + self.partition + '/' + 'transcript/'

        # List files in sefl.mfcc_dir using os.listdir in sorted order
        mfcc_names          = sorted(os.listdir(self.mfcc_dir)) 
        # List files in self.transcript_dir using os.listdir in sorted order
        transcript_names    = sorted(os.listdir(self.transcript_dir))
        # Making sure that we have the same no. of mfcc and transcripts
        assert len(mfcc_names) == len(transcript_names)

        max_index = math.floor(len(mfcc_names) * portion)
        self.mfccs, self.transcripts = [], []

        # Iterate through mfccs and transcripts
        for i in range(max_index):
        # for i in range(10):
        #   Load a single mfcc
            mfcc = np.load(self.mfcc_dir + mfcc_names[i])
        #   Do Cepstral Normalization of mfcc (explained in writeup)
            mfcc = mfcc - np.mean(mfcc, axis=0) / np.std(mfcc, axis=0)
        
        #   Load the corresponding transcript
            # Remove [SOS] and [EOS] from the transcript 
            # (Is there an efficient way to do this without traversing through the transcript?)
            # Note that SOS will always be in the starting and EOS at end, as the name suggests.
            transcript  = np.load(self.transcript_dir + transcript_names[i])[1:-1]

        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)        

        # NOTE:
        # Each mfcc is of shape T1 x 27, T2 x 27, ...
        # Each transcript is of shape (T1+2) x 27, (T2+2) x 27 before removing [SOS] and [EOS]

        # Concatenate all mfccs in self.mfccs such that 
        # the final shape is T x 27 (Where T = T1 + T2 + ...)
        self.mfccs = np.concatenate(self.mfccs, axis=0)
        self.mfccs = np.reshape(self.mfccs, (-1, 27))

        # Concatenate all transcripts in self.transcripts such that 
        # the final shape is (T,) meaning, each time step has one phoneme output
        self.transcripts = np.concatenate(self.transcripts, axis=0)
        self.transcripts = np.reshape(self.transcripts, (-1,))

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.mfccs)

        # Take some time to think about what we have done. 
        # self.mfcc is an array of the format (Frames x Features). 
        # Our goal is to recognize phonemes of each frame
        # From hw0, you will be knowing what context is. 
        # We can introduce context by padding zeros on top and bottom of self.mfcc
        # self.X = np.pad(self.X, ((context, context), (0, 0)), 'constant', constant_values= 0)
        self.mfccs = np.pad(self.mfccs, ((self.context, self.context), (0, 0)), 'constant', constant_values= 0)

        # The available phonemes in the transcript are of string data type
        # But the neural network cannot predict strings as such. 
        # Hence, we map these phonemes to integers
        # Map the phonemes to their corresponding list indexes in self.phonemes
        tokens = range(len(self.phonemes))
        self.VOCABULARY = dict(zip(self.phonemes, tokens))
        ### Map Labels To Tokens
        self.transcripts = [self.VOCABULARY[phoneme] for phoneme in self.transcripts]
        # Now, if an element in self.transcript is 0, it means that it is 'SIL' (as per the above example)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        # Based on context and offset, return a frame at given index with context frames to the left, and right.
        # After slicing, you get an array of shape 2*context+1 x 27. But our MLP needs 1d data and not 2d.
        # Flatten to get 1d data
        frames = self.mfccs[ind:ind+2*self.context+1, :].flatten()
        # print(frames.shape)
        frames = torch.FloatTensor(frames)
        phonemes = torch.tensor(self.transcripts[ind])       

        return frames, phonemes

In [140]:
class AudioTestDataset(torch.utils.data.Dataset):    
    # TODO: Create a test dataset class similar to the previous class but you dont have transcripts for this
    # Imp: Read the mfccs in sorted order, do NOT shuffle the data here or in your dataloader.
    def __init__(self, root, context, phonemes, partition): # Feel free to add more arguments

      self.context = context
      self.phonemes = phonemes
      self.root = root
      
      # MFCC directory - use partition to acces train/dev directories from kaggle data using root
      self.mfcc_dir       = self.root + partition + '/' + 'mfcc/'

      # List files in sefl.mfcc_dir using os.listdir in sorted order
      mfcc_names          = sorted(os.listdir(self.mfcc_dir)) 

      self.mfccs = []

      # Iterate through mfccs and transcripts
      for i in range(len(mfcc_names)):
      # for i in range(10):
      #   Load a single mfcc
          mfcc = np.load(self.mfcc_dir + mfcc_names[i])
      #   Do Cepstral Normalization of mfcc (explained in writeup)
          mfcc = mfcc - np.mean(mfcc, axis=0) / np.std(mfcc, axis=0)

      #   Append each mfcc to self.mfcc
          self.mfccs.append(mfcc)

      # NOTE:
      # Each mfcc is of shape T1 x 27, T2 x 27, ...

      # Concatenate all mfccs in self.mfccs such that 
      # the final shape is T x 27 (Where T = T1 + T2 + ...)
      self.mfccs = np.concatenate(self.mfccs, axis=0)
      self.mfccs = np.reshape(self.mfccs, (-1, 27))
      # print(self.mfccs.shape)

      # Hint: Use numpy to concatenate
      # Length of the dataset is now the length of concatenated mfccs/transcripts
      self.length = len(self.mfccs)

      # Take some time to think about what we have done. 
      # self.mfcc is an array of the format (Frames x Features). 
      # Our goal is to recognize phonemes of each frame
      # From hw0, you will be knowing what context is. 
      # We can introduce context by padding zeros on top and bottom of self.mfcc
      # self.X = np.pad(self.X, ((context, context), (0, 0)), 'constant', constant_values= 0)
      self.mfccs = np.pad(self.mfccs, ((self.context, self.context), (0, 0)), 'constant', constant_values= 0)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        # Based on context and offset, return a frame at given index with context frames to the left, and right.
        # After slicing, you get an array of shape 2*context+1 x 27. But our MLP needs 1d data and not 2d.
        # Flatten to get 1d data
        frames = self.mfccs[ind:ind+2*self.context+1, :].flatten()
        frames = torch.FloatTensor(frames)

        return frames

# Parameters Configuration

Storing your parameters and hyperparameters in a single configuration dictionary makes it easier to keep track of them during each experiment. It can also be used with weights and biases to log your parameters for each experiment and keep track of them across multiple experiments. 

In [125]:
"""
Paul's Architecture 1 = 60.58%
"""

# config = {
#     'epochs'        : 30,
#     'batch_size'    : 1024,
#     'context'       : 30,
#     'init_lr'       : 0.0005,
#     'architecture'  : 'very-low-cutoff',
#     'train_dataset' : 'train-clean-100',
#     'val_dataset'   : 'dev-clean',
#     'test_dataset'  : 'test-clean',
#     'dropout_rate'  : 0.15,
#     'weight_decay'  : 1e-2,
#     'step_size'     : 7
#     # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
# }

In [151]:
"""
Thomas's 75% Architecture 1 = %
"""

config = {
    'epochs'        : 5,
    'batch_size'    : 1024,
    'context'       : 10,
    'init_lr'       : 1e-4,
    'architecture'  : 'low-cutoff',
    'train_dataset' : 'train-clean-100',
    'val_dataset'   : 'dev-clean',
    'test_dataset'  : 'test-clean',
    'dropout_rate'  : 0,
    'weight_decay'  : 1e-2,
    'step_size'     : 1
    # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
}

# Create Datasets

In [142]:
#TODO: Create a dataset object using the AudioDataset class for the training data 
root_dir = '/content/data/11-785-s23-hw1p2/'
train_data = AudioDataset(root=root_dir, partition=config['train_dataset'], phonemes=PHONEMES, context=config['context'], portion=0.2)

# # TODO: Create a dataset object using the AudioDataset class for the validation data 
val_data = AudioDataset(root=root_dir, partition=config['val_dataset'], phonemes=PHONEMES, context=config['context'])

# # TODO: Create a dataset object using the AudioTestDataset class for the test data 
test_data = AudioTestDataset(root=root_dir, partition=config['test_dataset'], phonemes=PHONEMES, context=config['context'])

In [143]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
# We shuffle train dataloader but not val & test dataloader. Why?

train_loader = torch.utils.data.DataLoader(
    dataset     = train_data, 
    num_workers = 4,
    batch_size  = config['batch_size'], 
    pin_memory  = True,
    shuffle     = True
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data, 
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

test_loader = torch.utils.data.DataLoader(
    dataset     = test_data, 
    num_workers = 2, 
    batch_size  = config['batch_size'], 
    pin_memory  = True, 
    shuffle     = False
)


print("Batch size     : ", config['batch_size'])
print("Context        : ", config['context'])
print("Input size     : ", (2*config['context']+1)*27)
print("Output symbols : ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size     :  1024
Context        :  10
Input size     :  567
Output symbols :  42
Train dataset samples = 7033850, batches = 6869
Validation dataset samples = 1928204, batches = 1884
Test dataset samples = 1934138, batches = 1889


In [145]:
# Testing code to check if your data loaders are working
for i, data in enumerate(val_loader):
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    break

torch.Size([1024, 567]) torch.Size([1024])


In [146]:
# Testing code to check if your data loaders are working
val_phonemes = []
for i, val_data in enumerate(val_loader):
    val_frames, val_phoneme = val_data
    val_phonemes.append(val_phoneme.cpu().numpy())

# print(val_phonemes[0])
val_phonemes = np.concatenate(val_phonemes)
print(np.unique(val_phonemes, return_counts=True))
sil_index = np.argmax(np.unique(val_phonemes, return_counts=True)[1])
sil_count = np.max(np.unique(val_phonemes, return_counts=True)[1])
low_index = np.argmin(np.unique(val_phonemes, return_counts=True)[1])
low_count = np.min(np.unique(val_phonemes, return_counts=True)[1])
print(PHONEMES[sil_index])
print(sil_count)
print(PHONEMES[low_index])
print(low_count)


Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f464dbd7160><function _MultiProcessingDataLoaderIter.__del__ at 0x7f464dbd7160>

Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1466, in __del__
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1466, in __del__
        self._shutdown_workers()self._shutdown_workers()

  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1449, in _shutdown_workers
  File "/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py", line 1449, in _shutdown_workers
    if w.is_alive():
      File "/usr/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
if w.is_alive():
  File "/usr/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
        assert self._parent_pid == os.getpid(), 'can only test a c

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39]), array([319908,  29688,  49298, 123734,  29340,  20274,  49332,  23607,
        12644,  62763,  37100,  47112,  54928,  36184,  37562,  13541,
        34813,  74887,  70861,   8730,  47016,  65902,  44728,  94541,
        19327,  30755,   3861,  34131,  62686, 101184,  17628,  97390,
         9247,   6286,  26691,  27440,  37697,   9669,  54850,    869]))
[SIL]
319908
ZH
869


# Network Architecture


This section defines your network architecture for the homework. We have given you a sample architecture that can easily clear the very low cutoff for the early submission deadline.

In [133]:
"""
Paul's Architecture - 1 = 60.8%
"""

# # This architecture will make you cross the very low cutoff
# # However, you need to run a lot of experiments to cross the medium or high cutoff
# class Network(torch.nn.Module):

#     def __init__(self, input_size, output_size, dropout_rate):

#         super(Network, self).__init__()

#         self.model = torch.nn.Sequential(
#             torch.nn.Linear(input_size, 3918),
#             torch.nn.BatchNorm1d(3918),
#             torch.nn.GELU(),
#             torch.nn.Linear(3918, 1136),
#             torch.nn.BatchNorm1d(1136),
#             torch.nn.GELU(),
#             torch.nn.Dropout(p=dropout_rate),
#             torch.nn.Linear(1136, 3168),
#             torch.nn.BatchNorm1d(3168),
#             torch.nn.GELU(),
#             torch.nn.Dropout(p=dropout_rate),
#             torch.nn.Linear(3168, 869),
#             torch.nn.BatchNorm1d(869),
#             torch.nn.GELU(),
#             torch.nn.Dropout(p=dropout_rate),
#             torch.nn.Linear(869, 240),
#             torch.nn.BatchNorm1d(240),
#             torch.nn.GELU(),
#             torch.nn.Dropout(p=dropout_rate),
#             torch.nn.Linear(240, 167),
#             torch.nn.BatchNorm1d(167),
#             torch.nn.GELU(),
#             torch.nn.Dropout(p=dropout_rate),
#             torch.nn.Linear(167, output_size)
#         )
#         self.apply(self._init_weights)

#     def _init_weights(self, m):
#       if isinstance(m, torch.nn.Linear):
#         torch.nn.init.kaiming_uniform(m.weight)
#         torch.nn.init.zeros_(m.bias)

#     def forward(self, x):
#         out = self.model(x)

#         return out

"\nPaul's Architecture - 1 = 60.8%\n"

In [148]:
"""
Thomas's 75% Architecture 1 = %
"""
# This architecture will make you cross the very low cutoff
# However, you need to run a lot of experiments to cross the medium or high cutoff
class Network(torch.nn.Module):

    def __init__(self, input_size, output_size, dropout_rate):

        super(Network, self).__init__()
        
        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 1536),
            torch.nn.BatchNorm1d(1536),
            torch.nn.GELU(), # Layer 1
            torch.nn.Linear(1536, 1536),
            torch.nn.BatchNorm1d(1536),
            torch.nn.GELU(), # Layer 2
            torch.nn.Linear(1536, 1536),
            torch.nn.BatchNorm1d(1536),
            torch.nn.GELU(), # Layer 3
            torch.nn.Linear(1536, 1536),
            torch.nn.BatchNorm1d(1536),
            torch.nn.GELU(), # Layer 4
            torch.nn.Linear(1536, 1536),
            torch.nn.BatchNorm1d(1536),
            torch.nn.GELU(), # Layer 5
            torch.nn.Linear(1536, 1536),
            torch.nn.BatchNorm1d(1536),
            torch.nn.GELU(), # Layer 6
            torch.nn.Linear(1536, 1536),
            torch.nn.BatchNorm1d(1536),
            torch.nn.GELU(), # Layer 7
            torch.nn.Linear(1536, 1536),
            torch.nn.BatchNorm1d(1536),
            torch.nn.GELU(), # Layer 8
            torch.nn.Linear(1536, output_size))
        
        #self.apply(self._init_weights)

    #def _init_weights(self, m):
    #  if isinstance(m, torch.nn.Linear):
    #      torch.nn.init.normal_(m.weight, mean=0, std=0.0001)
    #      torch.nn.init.zeros_(m.bias)
        
    def forward(self, x):
        out = self.model(x)

        return out

In [132]:
# # This architecture will make you cross the very low cutoff
# # However, you need to run a lot of experiments to cross the medium or high cutoff
# class Network(torch.nn.Module):

#     def __init__(self, input_size, output_size, dropout_rate):

#         super(Network, self).__init__()

#         self.model = torch.nn.Sequential(
#             torch.nn.Linear(input_size, 3918),
#             torch.nn.BatchNorm1d(3918),
#             torch.nn.GELU(),
#             torch.nn.Linear(3918, 1136),
#             torch.nn.BatchNorm1d(1136),
#             torch.nn.GELU(),
#             torch.nn.Dropout(p=dropout_rate),
#             torch.nn.Linear(1136, 3168),
#             torch.nn.BatchNorm1d(3168),
#             torch.nn.GELU(),
#             torch.nn.Dropout(p=dropout_rate),
#             torch.nn.Linear(3168, 869),
#             torch.nn.BatchNorm1d(869),
#             torch.nn.GELU(),
#             torch.nn.Dropout(p=dropout_rate),
#             torch.nn.Linear(869, 240),
#             torch.nn.BatchNorm1d(240),
#             torch.nn.GELU(),
#             torch.nn.Dropout(p=dropout_rate),
#             torch.nn.Linear(240, 167),
#             torch.nn.BatchNorm1d(167),
#             torch.nn.GELU(),
#             torch.nn.Dropout(p=dropout_rate),
#             torch.nn.Linear(167, output_size)
#         )
#         self.apply(self._init_weights)

#     def _init_weights(self, m):
#       if isinstance(m, torch.nn.Linear):
#         torch.nn.init.kaiming_uniform(m.weight)
#         torch.nn.init.zeros_(m.bias)

#     def forward(self, x):
#         out = self.model(x)

#         return out

# Define Model, Loss Function and Optimizer

Here we define the model, loss function, optimizer and optionally a learning rate scheduler. 

In [149]:
INPUT_SIZE  = (2*config['context'] + 1) * 27 # Why is this the case?
model       = Network(INPUT_SIZE, len(train_data.phonemes),dropout_rate=config['dropout_rate']).to(device)
summary(model, frames.to(device))
# summary(model, frames.to(device))
# Check number of parameters of your network
# Remember, you are limited to 20 million parameters for HW1 (including ensembles)

                         Kernel Shape  Output Shape     Params  Mult-Adds
Layer                                                                    
0_model.Linear_0          [567, 1536]  [1024, 1536]   872.448k   870.912k
1_model.BatchNorm1d_1          [1536]  [1024, 1536]     3.072k     1.536k
2_model.GELU_2                      -  [1024, 1536]          -          -
3_model.Linear_3         [1536, 1536]  [1024, 1536]  2.360832M  2.359296M
4_model.BatchNorm1d_4          [1536]  [1024, 1536]     3.072k     1.536k
5_model.GELU_5                      -  [1024, 1536]          -          -
6_model.Linear_6         [1536, 1536]  [1024, 1536]  2.360832M  2.359296M
7_model.BatchNorm1d_7          [1536]  [1024, 1536]     3.072k     1.536k
8_model.GELU_8                      -  [1024, 1536]          -          -
9_model.Linear_9         [1536, 1536]  [1024, 1536]  2.360832M  2.359296M
10_model.BatchNorm1d_10        [1536]  [1024, 1536]     3.072k     1.536k
11_model.GELU_11                    - 

/usr/local/lib/python3.8/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_model.Linear_0,"[567, 1536]","[1024, 1536]",872448.0,870912.0
1_model.BatchNorm1d_1,[1536],"[1024, 1536]",3072.0,1536.0
2_model.GELU_2,-,"[1024, 1536]",NaN,NaN
3_model.Linear_3,"[1536, 1536]","[1024, 1536]",2360832.0,2359296.0
4_model.BatchNorm1d_4,[1536],"[1024, 1536]",3072.0,1536.0
5_model.GELU_5,-,"[1024, 1536]",NaN,NaN
6_model.Linear_6,"[1536, 1536]","[1024, 1536]",2360832.0,2359296.0
7_model.BatchNorm1d_7,[1536],"[1024, 1536]",3072.0,1536.0
8_model.GELU_8,-,"[1024, 1536]",NaN,NaN


In [152]:
criterion = torch.nn.CrossEntropyLoss() # Defining Loss function. 
# We use CE because the task is multi-class classification 

optimizer = torch.optim.Adam(model.parameters(), lr= config['init_lr'], weight_decay=config['weight_decay']) #Defining Optimizer
# Recommended : Define Scheduler for Learning Rate, 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=config['step_size'], gamma=0.1)
# including but not limited to StepLR, MultiStepLR, CosineAnnealingLR, ReduceLROnPlateau, etc. 
# You can refer to Pytorch documentation for more information on how to use them.

# Is your training time very high? 
# Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it 
# Refer - https://pytorch.org/docs/stable/notes/amp_examples.html

# Training and Validation Functions

This section covers the training, and validation functions for each epoch of running your experiment with a given model architecture. The code has been provided to you, but we recommend going through the comments to understand the workflow to enable you to write these loops for future HWs.

In [153]:
torch.cuda.empty_cache()
gc.collect()

2711

In [154]:
def train(model, dataloader, optimizer, criterion):

    model.train()
    tloss, tacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')
    
    for i, (frames, phonemes) in enumerate(dataloader):
        
        ### Initialize Gradients
        optimizer.zero_grad()

        ### Move Data to Device (Ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        ### Forward Propagation
        logits  = model(frames)

        ### Loss Calculation
        loss    = criterion(logits, phonemes)

        ### Backward Propagation
        loss.backward() 
        
        ### Gradient Descent
        optimizer.step()       

        tloss   += loss.item()
        tacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        batch_bar.set_postfix(loss="{:.04f}".format(float(tloss / (i + 1))), 
                              acc="{:.04f}%".format(float(tacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()
  
    batch_bar.close()
    tloss   /= len(train_loader)
    tacc    /= len(train_loader)

    return tloss, tacc

In [155]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode
    vloss, vacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Move data to device (ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        # makes sure that there are no gradients computed as we are not training the model now
        with torch.inference_mode(): 
            ### Forward Propagation
            logits  = model(frames)
            ### Loss Calculation
            loss    = criterion(logits, phonemes)

        vloss   += loss.item()
        vacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]
        
        # Do you think we need loss.backward() and optimizer.step() here?

        batch_bar.set_postfix(loss="{:.04f}".format(float(vloss / (i + 1))), 
                              acc="{:.04f}%".format(float(vacc*100 / (i + 1))))
        batch_bar.update()
    
        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    vloss   /= len(val_loader)
    vacc    /= len(val_loader)

    return vloss, vacc

# Weights and Biases Setup

This section is to enable logging metrics and files with Weights and Biases. Please refer to wandb documentationa and recitation 0 that covers the use of weights and biases for logging, hyperparameter tuning and monitoring your runs for your homeworks. Using this tool makes it very easy to show results when submitting your code and models for homeworks, and also extremely useful for study groups to organize and run ablations under a single team in wandb. 

We have written code for you to make use of it out of the box, so that you start using wandb for all your HWs from the beginning.

In [158]:
wandb.login(key="a948a7a6118c9cb14a779f2fbfbf9f4ea1b82754") #API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [159]:
# Create your wandb run
run = wandb.init(
    name    = "thomas-train75", ### Wandb creates random run names if you skip this field, we recommend you give useful names
    reinit  = True, ### Allows reinitalizing runs when you re-run this cell
    id     = "hw1p2-mark2", ### Insert specific run id here if you want to resume a previous run
    # resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw1p2", ### Project should be created in your wandb account 
    config  = config ### Wandb Config for your run
)

In [160]:
### Save your model architecture as a string with str(model) 
model_arch  = str(model)

### Save it in a txt file 
arch_file   = open("model_arch.txt", "w")
file_write  = arch_file.write(model_arch)
arch_file.close()

### log it in your wandb run with wandb.save()
wandb.save('model_arch.txt')

['/content/wandb/run-20230211_234615-hw1p2-mark2/files/model_arch.txt']

# Experiment

Now, it is time to finally run your ablations! Have fun!

In [161]:
# Iterate over number of epochs to train and evaluate your model
torch.cuda.empty_cache()
gc.collect()
wandb.watch(model, log="all")
best_val_acc = 0

for epoch in range(config['epochs']):

    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    curr_lr                 = float(optimizer.param_groups[0]['lr'])
    train_loss, train_acc   = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc       = eval(model, val_loader)

    print("\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc*100, train_loss, curr_lr))
    print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc*100, val_loss))

    scheduler.step()

    ### Log metrics at each epoch in your run 
    # Optionally, you can log at each batch inside train/eval functions 
    # (explore wandb documentation/wandb recitation)
    wandb.log({'train_acc': train_acc*100, 'train_loss': train_loss, 
               'val_acc': val_acc*100, 'valid_loss': val_loss, 'lr': curr_lr})

    ### Highly Recommended: Save checkpoint in drive and/or wandb if accuracy is better than your current best
    torch.save({'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            #'scheduler_state_dict': scheduler.state_dict(),
            'train_loss': train_loss,
            'val_acc': val_acc},
          '/content/drive/MyDrive/11-785-IDL/HW/HW1/P2/model_checkpoint.pth')

### Finish your wandb run
run.finish()


Epoch 1/5


Train:   0%|          | 0/6869 [00:00<?, ?it/s]

Val:   0%|          | 0/1884 [00:00<?, ?it/s]

	Train Acc 68.4189%	Train Loss 1.0232	 Learning Rate 0.0001000
	Val Acc 63.6206%	Val Loss 1.2102

Epoch 2/5


Train:   0%|          | 0/6869 [00:00<?, ?it/s]

...

	Train Acc 80.5613%	Train Loss 0.6464	 Learning Rate 0.0000001
	Val Acc 73.0045%	Val Loss 0.8915

Epoch 5/5


Train:   0%|          | 0/6869 [00:00<?, ?it/s]

Val:   0%|          | 0/1884 [00:00<?, ?it/s]

	Train Acc 80.6561%	Train Loss 0.6437	 Learning Rate 0.0000000
	Val Acc 72.9697%	Val Loss 0.8933


lr,█▂▁▁▁
train_acc,▁▆███
train_loss,█▃▁▁▁
val_acc,▁▇███
valid_loss,█▂▁▁▁
lr,0.0
train_acc,80.65612
train_loss,0.64373
val_acc,72.96969
valid_loss,0.89335


# Testing and submission to Kaggle

Before we get to the following code, make sure to see the format of submission given in *sample_submission.csv*. Once you have done so, it is time to fill the following function to complete your inference on test data. Refer the eval function from previous cells to get an idea of how to go about completing this function.

In [162]:
# Create your wandb run
run = wandb.init(
    name    = "thomas-test75", ### Wandb creates random run names if you skip this field, we recommend you give useful names
    reinit  = True, ### Allows reinitalizing runs when you re-run this cell
    id     = "hw1p2-mark2", ### Insert specific run id here if you want to resume a previous run
    # resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw1p2", ### Project should be created in your wandb account 
    config  = config ### Wandb Config for your run
)

In [164]:
def test(model, test_loader):
    ### What you call for model to perform inference?
    model.eval() # TODO train or eval?

    ### List to store predicted phonemes of test data
    test_predictions = []

    ### Which mode do you need to avoid gradients?
    with torch.inference_mode(): # TODO

        for i, mfccs in enumerate(tqdm(test_loader)):

            mfccs   = mfccs.to(device)             
            
            logits  = model(mfccs)

            ### Get most likely predicted phoneme with argmax
            predicted_phonemes = torch.argmax(logits, dim= 1).cpu().numpy()

            ### How do you store predicted_phonemes with test_predictions? Hint, look at eval 
            # TODO
            test_predictions.append(predicted_phonemes)

            ### Release memory
            del mfccs, logits
            torch.cuda.empty_cache()

    return np.concatenate(test_predictions).tolist()

In [ ]:
state_dict = torch.load('/content/drive/MyDrive/11-785-IDL/HW/HW1/P2/model_checkpoint.pth')
model.load_state_dict(state_dict['model_state_dict'])

In [165]:
predictions = test(model, test_loader)

  0%|          | 0/1889 [00:00<?, ?it/s]

In [166]:
### Create CSV file with predictions
with open("./submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
        f.write("{},{}\n".format(i, PHONEMES[predictions[i]]))
print(len(predictions))

1934138


In [167]:
### Submit to kaggle competition using kaggle API (Uncomment below to use)
!kaggle competitions submit -c 11-785-s23-hw1p2 -f ./submission.csv -m "Test Submission"

### However, its always safer to download the csv file and then upload to kaggle

100% 19.3M/19.3M [00:01<00:00, 12.5MB/s]
Successfully submitted to Frame-Level Speech Recognition